In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy.signal
from utils import audio, channels, synchronize, channel_estimate,ofdm, encode, decode, preamble,correction

cwd = os.getcwd()

In [ ]:
#baisc parameter definition
filename = 'self_test_06_31_53'            #rememeber to change everytime

chirp_duration = 1
record_duration = 20
fs = 48000

DFT_LENGTH = 4096
CP_LENGTH = 512
symbol_per_frame = 128
low_freq = 1000
high_freq = 10000

encode_method = 'qpsk'
double_chirp = False
known_ofdm = True

#define chirp used in the signal
chirp_range = [1000,10000]
chirp = preamble.generate_chirp(chirp_duration, fs, low=chirp_range[0], high=chirp_range[1], silence_duration=0, double=double_chirp)

#define known ofdm symbol
known_ofdm = True
repeat_time = 4
known_ofdm_data = preamble.load_known_ofdm(CP_LENGTH,repeat_time)
spb = ofdm.subcarriers_per_block(fs,DFT_LENGTH,low_freq,high_freq)

In [ ]:
# load recorded signals
received_signal,_ = audio.wav_to_signal(filename,'audio_files')
# received_signal = np.load("rec.npy")

In [ ]:
#synchronize: convolve and find
start,convolved = synchronize.chirp_synchronize(received_signal,chirp_range, fs,duration=1)
event_end = np.array(synchronize.impulse_detect(abs(convolved),fs,duration = 1,window_time=0.3,threshold=2))

In [ ]:
#plot the convolved result
plt.plot(convolved)
for pos in event_end:
    plt.axvline(x = pos, color = 'r')
plt.show()

In [ ]:
# #check that the synchronization is correct
# a1 = np.argmax(abs(convolved[:record_duration*fs//2]))
# a2 = np.argmax(abs(convolved[record_duration*fs//2:])) + record_duration*fs//2

# print(event_end,"max of convoled data: ", a1,a2)

In [ ]:
#plot found chirp in received signal
event_start = event_end - fs*chirp_duration
chirp_received = received_signal[event_start[0]:event_start[0]+chirp_duration*fs]   #taking only the first event for the time being

plt.plot(received_signal)
for pos in event_start:
    plt.axvline(x = pos, color = 'r')
plt.show()
print(str(len(event_start))+" chirp events found at "+ str(event_start))

In [ ]:
event = event_start[1:3]
gap = fs * chirp_duration + 2 *CP_LENGTH + 2 * repeat_time * DFT_LENGTH + 128 * (CP_LENGTH + DFT_LENGTH)
print(event[1]-event[0]-gap) #376704 for 64 symbols
event = event -10

In [ ]:
#estimate with known ofdm
known_ofdm_start = int(event[0] + fs*(chirp_duration)) #need to change accordingly
known_ofdm_end = known_ofdm_start + known_ofdm_data.size
received_known = received_signal[known_ofdm_start:known_ofdm_end]  #received ofdm part

#remove the first cyclic prefix manually then find discarded
received_known = received_known[CP_LENGTH:]
#fft_known = ofdm.ofdm_to_fourier(received_known[:], DFT_LENGTH, cp_length=0) 
#discarded_known = ofdm.subcarrier_extract(fft_known, DFT_LENGTH, fs, low_freq, high_freq)

#estimate
# H_known_ofdm = ofdm.known_ofdm_estimate(discarded_known,repeat_time,known_ofdm_data[CP_LENGTH:],DFT_LENGTH,low_freq,high_freq,fs)
H_known_ofdm = ofdm.known_ofdm_estimate_edited(received_known,known_ofdm_data[CP_LENGTH:CP_LENGTH+DFT_LENGTH],DFT_LENGTH,CP_LENGTH,low_freq,high_freq,fs)
plt.plot(np.abs(H_known_ofdm))
plt.show()
plt.plot(np.angle(H_known_ofdm))
plt.show()

In [ ]:
#used for ofdm after a chirp only
ofdm_start = int(known_ofdm_end)
#end = received_signal.size      #end of the possible range with a signal
#correction = (end-ofdm_start)%(DFT_LENGTH+CP_LENGTH)
#ofdm_end = end - correction
ofdm_end = ofdm_start+symbol_per_frame*(DFT_LENGTH+CP_LENGTH)       #hard coded, need to check
received_ofdm =received_signal[ofdm_start:ofdm_end]  #received ofdm information
fft = ofdm.ofdm_to_fourier(received_ofdm, DFT_LENGTH, CP_LENGTH)
discarded = ofdm.subcarrier_extract(fft, DFT_LENGTH, fs, low_freq, high_freq)
deconvolved = ofdm.deconvolve(discarded, H_known_ofdm, DFT_LENGTH, fs, low_freq, high_freq,True)

In [ ]:
# phase correction
sample_diff = event[1]-event[0]-gap
sample_corrected = correction.phase_correction(deconvolved, sample_diff, DFT_LENGTH, CP_LENGTH, fs, low_freq, high_freq, chirp_duration)

In [ ]:
#plot the phase difference between the two channel estimation done with known ofdm symbols
phase_diff,H1,H2 = correction.phase_difference(received_signal, event,known_ofdm_data,CP_LENGTH,DFT_LENGTH,fs,low_freq,high_freq,repeat_time)

In [ ]:
#find regression of the phase diff plot by looking at a selected linear range
#iterate until satisfied with range for regression

from IPython.display import clear_output
reply = "n"
while reply != "y":
    regression_range_1 = np.array(input("range to perform regression: ").split()).astype(int)
    clear_output()
    start1,end1 = regression_range_1
    if end1 > phase_diff.size:
        end1 = phase_diff.size
    plt.plot(phase_diff[start1:end1])
    section1 = phase_diff[start1:end1]
    x_1 = np.linspace(0,end1,num=(end1-start1))
    slope1, intercept1, r_value, p_value, std_err = scipy.stats.linregress(x_1, section1)
    plt.plot(x_1*slope1+intercept1)
    print(slope1,intercept1)
    plt.title("1st regression")
    plt.show()
    reply = input("confirm range? y/n")

clear_output()
regression_corrected = correction.regression_correction(spb,slope1,intercept1,H1,H2,sample_corrected,symbol_per_frame)



In [ ]:
#check accuracy
new = True
if new:
    np.random.seed(8)
    encoded = np.random.randint(2, size=2*128*spb)
    np.savetxt('encoded', encoded, fmt='%i')


In [ ]:
corrected = regression_corrected

phase_plot_range = 3000
plt.scatter(np.real(corrected[:phase_plot_range]),np.imag(corrected[:phase_plot_range])) #plot the received phase distribution
print(corrected.size)

In [ ]:
decoded = decode.qpsk_decode(corrected)

print(encoded.size, decoded.size)
print("source:\n", encoded[:30])
print("decoded:\n", decoded[:30])

In [ ]:
equality = encoded == decoded[:encoded.size]
equality = equality[:]
print("correct rate:\n", np.sum(equality.astype(int))/equality.size)

In [ ]:
#error of each symbol
e_symbol = [0]*symbol_per_frame
for n in range(symbol_per_frame):
    n=int(n)
    a = equality[2*spb*n:2*spb*(n+1)]
    e_symbol[n] = np.round(np.sum(a.astype(int))/a.size,4)
plt.plot(e_symbol)

In [ ]:
print(e_symbol[0])